In [1]:
import pandas as pd
import numpy as np
import yaml
import CausalEGM as cegm
import evaluationData

2024-04-01 13:17:04.540020: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-01 13:17:04.910715: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-01 13:17:05.552466: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 13:17:05.552502: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 13:17:05.554474: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

## BPIC 2017 Dataset

In [2]:
df = pd.read_csv("bpi2017_final.csv")
print(df.columns)
df.head()

Index(['case:concept:name', 'NumberOfOffers', 'Action', 'org:resource',
       'concept:name', 'EventOrigin', 'lifecycle:transition', 'time:timestamp',
       'case:LoanGoal', 'case:ApplicationType', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'Selected', 'CreditScore', 'OfferedAmount', 'treatedCase',
       'caseSuccesful', 'treatmentSuccess', 'offerNumber', 'offerSuccess',
       'treatmentOffer', 'timeApplication', 'weekdayApplication'],
      dtype='object')


,case:concept:name,NumberOfOffers,Action,org:resource,concept:name,EventOrigin,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,...,CreditScore,OfferedAmount,treatedCase,caseSuccesful,treatmentSuccess,offerNumber,offerSuccess,treatmentOffer,timeApplication,weekdayApplication
0,0.0,1.0,0.0,0.0,4.0,0.0,1.0,651433.0,10.0,1.0,...,0.0,5000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000,2.0
1,0.0,1.0,4.0,0.0,8.0,0.0,1.0,651434.0,10.0,1.0,...,0.0,5000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.061,2.0
2,0.0,1.0,0.0,0.0,22.0,2.0,3.0,651435.0,10.0,1.0,...,0.0,5000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.290,2.0
3,0.0,1.0,1.0,0.0,22.0,2.0,6.0,651437.0,10.0,1.0,...,0.0,5000.0,0.0,0.0,0.0,1.0,0.0,0.0,66.613,2.0
4,0.0,1.0,0.0,0.0,21.0,2.0,3.0,651438.0,10.0,1.0,...,0.0,5000.0,0.0,0.0,0.0,1.0,0.0,0.0,66.620,2.0


In [3]:
feature_names = ['NumberOfOffers', 'Action', 'org:resource',
       'concept:name', 'EventOrigin', 'lifecycle:transition', 'time:timestamp',
       'case:LoanGoal', 'case:ApplicationType', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'CreditScore', 'OfferedAmount', 'offerNumber','timeApplication', 'weekdayApplication']

In [4]:
params = yaml.safe_load(open('egm-nn/bpi2017.yaml', 'r'))
print(params)

{'dataset': 'BPI2017', 'output_dir': './egm-nn/', 'v_dim': 19, 'z_dims': [3, 3, 6, 6], 'x_min': 0, 'x_max': 3, 'lr': 0.0002, 'alpha': 1, 'beta': 1, 'gamma': 10, 'g_d_freq': 5, 'g_units': [64, 64, 64, 64, 64], 'e_units': [64, 64, 64, 64, 64], 'f_units': [64, 32, 8], 'h_units': [64, 32, 8], 'dz_units': [64, 32, 8], 'dv_units': [64, 32, 8], 'binary_treatment': True, 'use_z_rec': True, 'use_v_gan': True, 'save_model': False, 'save_res': True}


In [5]:
model = cegm.CausalEGM(params=params,random_seed=123)

In [6]:
#treatment (x), potential outcome (y), and covariates (v)
x = df['treatmentOffer']
y = df['offerSuccess']
v = df[feature_names]

In [9]:
model.train(data=[x,y,v],n_iter=100,save_format='npy',verbose=False)

The average treatment effect (ATE) is  0.16055138


In [10]:
#results = np.load('./egm-nn/results/BPI2017/20240122_162745/causal_pre_final.npy')
results = np.load('./egm-nn/results/BPI2017/20240401_131752/causal_pre_final.npy')
results

array([[0.07128906],
       [0.06054688],
       [0.02050781],
       ...,
       [0.22265625],
       [0.2734375 ],
       [0.2265625 ]], dtype=float32)

In [12]:
ite = results.reshape(-1)
ate = ite.mean()

In [13]:
# Calculate statistics
data = np.reshape(results, -1)
minimum = np.min(data)
first_quartile = np.percentile(data, 25)
median = np.median(data)
third_quartile = np.percentile(data, 75)
maximum = np.max(data)

# Interquartile range (IQR)
iqr = third_quartile - first_quartile

# Define upper and lower bounds for outliers
upper_bound = third_quartile + 1.5 * iqr
lower_bound = first_quartile - 1.5 * iqr

# Detect outliers
outliers = data[(data < lower_bound) | (data > upper_bound)]

ite_egm = [minimum, first_quartile, median, third_quartile, maximum, iqr, upper_bound, lower_bound]
print(ite_egm)

[-0.5625, 0.07421875, 0.16796875, 0.236328125, 0.875, 0.162109375, 0.4794921875, -0.1689453125]


In [31]:
#Store result in df of the results
%store -r df_results
lib = "CausalEGM"
method = "CausalEGM"

if method in df_results['method'].values:
    df_results.loc[df_results['method'] == method, 'ATE'] = ate
    index = df_results[df_results['method'] == method].index[0]
    df_results.at[index, 'ITE'] = ite_egm

else:
    df_results = df_results._append({'method': method, 'ATE': ate, 'ITE': ite_egm, 'Library': lib}, ignore_index=True)

%store df_results

Stored 'df_results' (DataFrame)


## Synthetic Dataset

In [2]:
df_synth = pd.read_csv("synthetic_dataset.csv")
synthetic_features = ['NumberOfOffers', 'concept:name',
       'lifecycle:transition', 'time:timestamp', 'elementId', 'resourceId',
       'weekdayApplication', 'timeApplication']
df_synth.head()

,case:concept:name,NumberOfOffers,concept:name,lifecycle:transition,time:timestamp,elementId,resourceId,treatment,successful,treatmentSuccess,weekdayApplication,timeApplication
0,0.0,2.0,29.0,0.0,152.0,28.0,1.0,1.0,1.0,2.0,1.0,0.0
1,0.0,2.0,29.0,2.0,152.0,28.0,1.0,1.0,1.0,2.0,1.0,0.0
2,0.0,2.0,29.0,1.0,152.0,28.0,1.0,1.0,1.0,2.0,1.0,0.0
3,0.0,2.0,5.0,0.0,152.0,7.0,1.0,1.0,1.0,2.0,1.0,0.0
4,0.0,2.0,5.0,2.0,152.0,7.0,0.0,1.0,1.0,2.0,1.0,0.0


In [11]:
params_synth = yaml.safe_load(open('egm-nn/synthetic-Dataset.yaml', 'r'))
print(params_synth)

{'dataset': 'SynthData', 'output_dir': './egm-nn/', 'v_dim': 8, 'z_dims': [3, 3, 6, 6], 'x_min': 0, 'x_max': 3, 'lr': 0.0002, 'alpha': 1, 'beta': 1, 'gamma': 10, 'g_d_freq': 5, 'g_units': [64, 64, 64, 64, 64], 'e_units': [64, 64, 64, 64, 64], 'f_units': [64, 32, 8], 'h_units': [64, 32, 8], 'dz_units': [64, 32, 8], 'dv_units': [64, 32, 8], 'binary_treatment': True, 'use_z_rec': True, 'use_v_gan': True, 'save_model': False, 'save_res': True}


In [8]:
#treatment (x), potential outcome (y), and covariates (v)
x = df_synth['treatment']
y = df_synth['treatmentSuccess']
v = df_synth[synthetic_features]

In [12]:
synth_model = cegm.CausalEGM(params=params_synth,random_seed=123)

In [13]:
synth_model.train(data=[x,y,v],n_iter=100,save_format='npy',verbose=False)

The average treatment effect (ATE) is  0.07459373


In [3]:
synth_results = np.load('./egm-nn/results/SynthData/20240227_122224/causal_pre_final.npy')
synth_results

array([[0.12651193],
       [0.11582285],
       [0.1216473 ],
       ...,
       [0.08596039],
       [0.0859375 ],
       [0.08587646]], dtype=float32)

In [5]:
synth_ite = synth_results.reshape(-1)
synth_ate = synth_results.mean()
print(synth_ite, synth_ate)

[0.12651193 0.11582285 0.1216473  ... 0.08596039 0.0859375  0.08587646] 0.07459373


In [6]:
import evaluation_metrics
true_ate = 2
boxplot = evaluation_metrics.boxplot_ite(synth_results)
metrics = evaluation_metrics.evaluation_metrics(true_ate, synth_results)
print(boxplot, metrics)

[-0.12484968, 0.059909820556640625, 0.07635641, 0.09584903717041016, 0.3586205, 0.03593921661376953, 0.14975786209106445, 0.006000995635986328] [3.708275, 1.9254065, 1.9256883, 1.9254065, 0.96284413]


In [7]:
%store -r df_synthetic_results_metric

method = "CausalEGM"
ate = synth_ate
ite = boxplot
metric = metrics

df_synthetic_results_metric = df_synthetic_results_metric._append({'method': method, 'ATE': ate, 'ITE': ite, 'metrics': metric}, ignore_index=True)

print(df_synthetic_results_metric)
%store df_synthetic_results_metric

                                           method  \
0                                    S-Learner LR   
1                                   XGBTRegressor   
2                              BaseTRegressor XGB   
3                               BaseTRegressor LR   
4                              BaseXRegressor XGB   
5                               BaseXRegressor LR   
6   BaseXRegressor XGB (without propensity score)   
7    BaseXRegressor LR (without propensity score)   
8                              BaseRRegressor XGB   
9                               BaseRRegressor LR   
10        BaseRRegressor XGB (with random weight)   
11  BaseRRegressor XGB (without propensity score)   
12                           Neural Network (MLP)   
13                                         BCAUSS   
14                                          CEVAE   
15             Accelerated Bayesian Causal Forest   
16                                      CausalEGM   

                                             

## Evaluation Dataset

In [4]:
import os
# Define folder path
folder_path = "./evaluationDatasets/Cause/"

params = yaml.safe_load(open('egm-nn/bpi2017.yaml', 'r'))
model = cegm.CausalEGM(params=params,random_seed=123)

# List to store treatment effects
ate_values = []

feature_names = ['NumberOfOffers', 'Action', 'org:resource',
       'concept:name', 'EventOrigin', 'lifecycle:transition', 'time:timestamp',
       'case:LoanGoal', 'case:ApplicationType', 'case:RequestedAmount',
       'FirstWithdrawalAmount', 'NumberOfTerms', 'Accepted', 'MonthlyCost',
       'CreditScore', 'OfferedAmount', 'offerNumber','timeApplication', 'weekdayApplication']

columns_to_drop = ['offerSuccess', 'treatmentOffer']

# Iterate through files in the folder
for file_name in os.listdir(folder_path):
    print(file_name)
    # Read CSV file
    file_path = os.path.join(folder_path, file_name)
    refutation = pd.read_csv(file_path)

    X = refutation[feature_names]
    # X = refutation.drop(columns=columns_to_drop)    
    y = refutation['offerSuccess']
    t = refutation['treatmentOffer']
    
    # params_cause = yaml.safe_load(open('egm-nn/bpi2017-CommonCause.yaml', 'r'))
    # model = cegm.CausalEGM(params=params_cause,random_seed=123)

    model.train(data=[t,y,X],n_iter=100, save_format='npy',verbose=False)

addRandomCauseDataset2.csv
The average treatment effect (ATE) is  0.12480156
addRandomCauseDataset1.csv
The average treatment effect (ATE) is  0.17548344
